In [15]:
!pip install fake-useragent

^C


In [45]:
#кравлинг
from fake_useragent import UserAgent
ua = UserAgent()
import sqlite3
from bs4 import BeautifulSoup
import re
import requests

session = requests.session()
page_number = 1
url = f'https://www.kinopoisk.ru/film/262970/reviews/ord/date/status/all/perpage/10/page{page_number}/'
req = session.get(url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
reviews = soup.find_all('div', {'class': 'reviewItem userReview'})
len(reviews)

10

In [46]:
import unicodedata

def parse_reviews_block(one_block):
    block = {}
    block['author'] = one_block.find('p', {'class':"profile_name"}).text.strip()
    
    if one_block.find('div', {'class':"response good"}) in one_block:
        block['sentiment'] = 'good'
    elif one_block.find('div', {'class':"response bad"}) in one_block:
        block['sentiment'] = 'bad'
    else:
        block['sentiment'] = 'neutral'
        
    block['pub_date'] = one_block.find('span', {'class':"date"}).text.strip()
    block['full_text'] = unicodedata.normalize("NFKD", one_block.find('table').text.strip())
    
    return block

In [48]:
def get_nth_page(page_number):
    
    session = requests.session()
    url = f'https://www.kinopoisk.ru/film/262970/reviews/ord/date/status/all/perpage/10/page/{page_number}/'
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('div', {'class': 'reviewItem userReview'})
    
    result = []
    for n in reviews:
        try:
            result.append(parse_reviews_block(n))
        except Exception as e:
            print(e)
    return result

In [57]:
from tqdm.auto import tqdm
import csv

def run_all(n_pages):
    with open('reviews.csv', 'a', newline='', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['author', 'sentiment', 'pub_date', 'full_text'])
        for i in tqdm(range(n_pages)):
            blocks = get_nth_page(i+1)
            writer.writeheader()
            writer.writerows(blocks)
            

In [154]:
run_all(20)

  0%|          | 0/20 [00:00<?, ?it/s]

In [156]:
#работа с файлом, подготовка текстов
import pandas as pd
corpus = pd.read_csv('reviews.csv')
df=pd.DataFrame(corpus)
df.tail()

,author,sentiment,pub_date,full_text
60,KRIpTOS,bad,27 марта 2009 | 15:18,Лично я не видел ничего чудовищнее! Этот сериа...
61,_MaLiNa_,good,19 февраля 2009 | 14:16,"Многие пишут, что этот сериал всего лишь тупая..."
62,levsky,good,03 февраля 2009 | 20:50,"Сразу скажу, что сериал 'Женаты и с детьми' я ..."
63,Фениксита,neutral,25 января 2009 | 16:48,Я не собираюсь никого осуждать или спорить. И ...
64,Ser_BarBoss,bad,09 января 2009 | 01:13,Первая серия. Первый сезон. Впервые мы знаком...


In [157]:
from pymystem3 import Mystem
f_bad=df['full_text'].loc[df['sentiment'] == 'bad'].values
with open("lemmas_reviews_bad.txt", 'w', encoding="utf-8") as file1:
    mystem = Mystem()
    for i in f_bad:
        lemmas = mystem.lemmatize(i)
        file1.write(''.join(lemmas))

f_good=df['full_text'].loc[df['sentiment'] == 'good'].values
with open("lemmas_reviews_good.txt", 'w', encoding="utf-8") as file2:
    mystem = Mystem()
    for i in f_good:
        lemmas = mystem.lemmatize(i)
        file2.write(''.join(lemmas))
        


In [158]:
import nltk
from nltk.tokenize import word_tokenize
import string

tokens_bad=[]
with open("lemmas_reviews_bad.txt", 'r', encoding="utf-8") as file1:
    for line in file1:
        tokens=word_tokenize(line.rstrip())
        for tok in tokens:
            if tok not in string.punctuation:
                tokens_bad.append(tok)
        
tokens_good=[]
with open("lemmas_reviews_good.txt", 'r', encoding="utf-8") as file2:
    for line in file2:
        tokens=word_tokenize(line.rstrip())
        for tok in tokens:
            if tok not in string.punctuation:
                tokens_good.append(tok)

In [207]:
from nltk.corpus import stopwords 
nltk.download('stopwords')
stops = set(stopwords.words('russian'))

final_tb = {}
final_tg = {}

for el in tokens_bad:
    if el not in tokens_good and el not in stops:
        if el not in final_tb:
            final_tb[el]=1
        else:
            final_tb[el]+=1
            
very_final_tb=[]
for keys in final_tb:
    if final_tb[keys]>2:
        very_final_tb.append(keys)
        
for el in tokens_good:
    if el not in tokens_bad and el not in stops:
        if el not in final_tg:
            final_tg[el]=1
        else:
            final_tg[el]+=1
            
very_final_tg=[]
for keys in final_tg:
    if final_tg[keys]>=2:
        very_final_tg.append(keys)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dimba\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [208]:
di={}

for i in very_final_tg:
    di[i]='good'
for i in very_final_tb:
    di[i]='bad'

In [209]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
X = count_vect.fit_transform(list(di.keys()))
X.shape

(236, 232)

In [210]:
X[0]

<1x232 sparse matrix of type '<class 'numpy.int64'>'
	with 1 stored elements in Compressed Sparse Row format>

In [211]:
from sklearn.naive_bayes import MultinomialNB

In [212]:
X[0].toarray()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int64)

In [213]:
clf = MultinomialNB()
y = list(di.values())
clf.fit(X, y) 

MultinomialNB()

In [214]:
clf.feature_log_prob_

array([[-5.29330482, -5.29330482, -5.29330482, -5.29330482, -5.29330482,
        -5.98645201, -5.29330482, -5.29330482, -5.98645201, -5.29330482,
        -5.29330482, -5.29330482, -5.29330482, -5.29330482, -5.29330482,
        -5.29330482, -5.98645201, -5.29330482, -5.98645201, -5.29330482,
        -5.98645201, -5.29330482, -5.29330482, -5.29330482, -5.98645201,
        -5.29330482, -5.98645201, -5.98645201, -5.98645201, -5.98645201,
        -5.98645201, -5.29330482, -5.98645201, -5.98645201, -5.29330482,
        -5.29330482, -5.29330482, -5.29330482, -5.29330482, -5.29330482,
        -5.29330482, -5.29330482, -5.29330482, -5.29330482, -5.29330482,
        -5.29330482, -5.29330482, -5.98645201, -5.29330482, -5.29330482,
        -5.29330482, -5.29330482, -5.29330482, -5.98645201, -5.98645201,
        -5.98645201, -5.29330482, -5.29330482, -5.98645201, -5.98645201,
        -5.29330482, -5.29330482, -5.29330482, -5.29330482, -5.98645201,
        -5.29330482, -5.29330482, -5.29330482, -5.2

In [215]:
import random
n = random.choice(range(64))
print(df['full_text'][n])

Хорошо, что сняли что-то необычное. Смешное! С веселыми шутками и актерами.

«Счастливы вместе» - комедия, наполненная красочными моментами и штуками, спецэффектами. Много напряженных моментов. Завораживает!

Даша Букина это, наверное, самая смешная из всех в семейке Букиных. Даже, несмотря на то, что она не работает. Очень необычно!

Светка Букина смешная девчонка с моральными принципами и образами. Очень интересный и красивый персонаж. Среди семьи. На нее интересно смотреть со стороны телевизора. Настоящая актриса!

Гена Букин продавец обуви, можно еще сказать любимец футбола! Мне очень смешно то, что он ничего не делает. Кроме как смотрит телевизор. Поверьте действительно странно!

И последний про кого хочется сказать Рома Букин настоящий парень с другими взглядами на мир и жизнь. Поклонник всех девочек! 

В целом  здесь все хорошо играют. Не могу сказать, что здесь есть что-то плохое. Одни шутки! Сказанные актерами! Комедия, в которой ничего нет грубого, плохого.

Вообще мне

In [216]:
clf.predict(X[n])

array(['good'], dtype='<U4')

In [ ]:
Возможно, для улучшения алгоритма можно использовать синонимичные слова или близкие по смыслу к тем,
которые мы обнаружили только в плохих или только в хороших отзывах. Например, при помощи Word2Vec и библиотеки Gensim